In [1]:
from utils import *
check_tensorflow_gpu()

proxy = SpatiotemporalCO2()
proxy.__dict__

c:\Users\Misael Morales\miniconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Misael Morales\miniconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you 

... Checking Tensorflow Version ...
Tensorflow built with CUDA? True
TF: 2.10.0 | CUDA: 64_112 | cuDNN: 64_8
# GPU available: 1 (NVIDIA GeForce RTX 3080)


{'input_features_dir': 'simulations2D/input_features',
 'output_targets_dir': 'simulations2D/output_targets',
 'n_realizations': 1000,
 'x_channels': 4,
 'y_channels': 2,
 'timesteps': 60,
 'dim': 64,
 'test_size': 0.25,
 't_samples': [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60],
 'optimizer': <tensorflow_addons.optimizers.weight_decay_optimizers.AdamW at 0x14d9b0a27a0>,
 'criterion': <bound method SpatiotemporalCO2.custom_loss of <utils.SpatiotemporalCO2 object at 0x0000014DC21145B0>>,
 'L1L2_split': 0.25,
 'loss_alpha': 0.75,
 'regular': <keras.regularizers.L1 at 0x14d9b894a00>,
 'leaky_slope': 0.25,
 'num_epochs': 100,
 'batch_size': 30,
 'lr_decay': 10,
 'verbose': 0}

In [2]:
proxy.load_data()
proxy.process_data(subsample=None)

X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
MinMax Normalization Done!
Train - X: (750, 64, 64, 4) | y: (750, 11, 64, 64, 2)
Test  - X: (250, 64, 64, 4) | y: (250, 11, 64, 64, 2)


In [3]:
model0, enc = proxy.make_model()
model0.summary()

Model: "CNN_RNN_Proxy"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 4)]  0           []                               
                                                                                                  
 separable_conv2d (SeparableCon  (None, 64, 64, 16)  116         ['input_1[0][0]']                
 v2D)                                                                                             
                                                                                                  
 squeeze_excite (SqueezeExcite)  (None, 64, 64, 16)  148         ['separable_conv2d[0][0]']       
                                                                                                  
 layer_normalization (LayerNorm  (None, 64, 64, 16)  32          ['squeeze_excite[0][0

In [4]:
model, fit = proxy.train(model0)

# Parameters: 6,460,518 | Batch size: 30 
Epoch: [10/100] - Loss: 0.1545 - Val Loss: 0.2062
Epoch: [20/100] - Loss: 0.0490 - Val Loss: 0.0888
Epoch: [30/100] - Loss: 0.0335 - Val Loss: 0.0593
Epoch: [40/100] - Loss: 0.0295 - Val Loss: 0.0531
Epoch: [50/100] - Loss: 0.0257 - Val Loss: 0.0484
Epoch: [60/100] - Loss: 0.0232 - Val Loss: 0.0492
Epoch: [70/100] - Loss: 0.0220 - Val Loss: 0.0595
Epoch: [80/100] - Loss: 0.0206 - Val Loss: 0.0666
Epoch: [90/100] - Loss: 0.0188 - Val Loss: 0.0419


In [ ]:
y_train_pred = model.predict(proxy.X_train)
y_test_pred  = model.predict(proxy.X_test)
print('Train pred: {} | Test pred: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
plot_results(180, proxy.X_train, proxy.y_train, y_train_pred, proxy.t_samples)

***
# END